# Convolutional Neural Networks

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from tensorflow.keras import layers, models, optimizers, losses, metrics, datasets

In [3]:
BATCH_SIZE = 128
ETA = 1e-3
EPOCHS = 10

다음 코드는 MNIST 데이터셋을 불러옵니다. 이때, `trainset`, `testset`은 다음과 같은 모양이 됩니다.
```python
trainset = (x_train, y_train)
testset = (x_test, y_test)
```

In [4]:
trainset, testset = datasets.mnist.load_data()

이것을 tensorflow dataset `tf.data.Dataset` 객체로 만들어줄 것입니다. `trainset`, `testset`은 numpy객체이고, `tf.data.Dataset`은 `.from_tensor_slices`라는 메소드를 제공하여 numpy array를 `tf.data.Dataset`으로 변환하는 기능을 제공합니다.

`tf.data.Dataset.from_tensor_slices()`는 튜플을 입력으로 받는데, 튜플은 `(data_array, label_array)`의 형태이어야 합니다. `mnist.load_data()`메소드가 이미 그런 형태로 `trainset`, `testset`을 생성해 주었으므로, 그냥 그대로 `.from_tensor_slices`에 인자로 넘겨주면 됩니다.

In [5]:
trainloader = tf.data.Dataset.from_tensor_slices(trainset).batch(BATCH_SIZE).shuffle(4096)
testloader = tf.data.Dataset.from_tensor_slices(testset).batch(BATCH_SIZE)

여기서, `.batch(BATCH_SIZE).shuffle(4096)`라는 문구가 추가되었네요. 이것을 알기 이전에, `tf.data.Dataset`을 좀 더 알 필요가 있는데요, `tf.data.Dataset`은 python의 generator입니다.
즉, `iter()`로 감쌀 수 있고, `next()`함수로 어떤 원소를 generate할 수 있는 객체입니다. `tf.data.Dataset`은 `next()`를 한 번 호출할 때마다 데이터 샘플 1개를 기본적으로 반환합니다. 하지만, `.batch(BATCH_SIZE)`라는 메소드를 호출해주면, `next()`를 한 번 호출해 줄 때마다 `BATCH_SIZE`개 만큼의 데이터를 반환해줍니다.

`.shuffle(4096)`의 경우, 1번의 epoch을 돌때마다 `tf.data.Dataset`안의 데이터를 셔플해주라는 의미입니다. `4096`은 그냥 버퍼크기로, 셔플의 속도에 아주 조금 차이가 있는 것 같습니다.

In [6]:
class ConvolutionalNeuralNetwork(models.Model):
    
    def __init__(self):
        super(ConvolutionalNeuralNetwork, self).__init__()
        
        self.features = models.Sequential([
            layers.Conv2D(16, (3, 3), strides=(1, 1), padding="SAME"),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.leaky_relu),
            
            layers.Conv2D(32, (3, 3), strides=(1, 1), padding="SAME"),
            layers.BatchNormalization(),
            layers.Activation(tf.nn.leaky_relu),
        ])
        
        self.flatten = layers.Flatten()
        
        self.classifier = models.Sequential([
            layers.Dense(32),
            layers.Activation(tf.nn.leaky_relu),
            layers.Dropout(0.5),
            
            layers.Dense(10),
            layers.Activation(tf.nn.softmax)
        ])
        
    def __call__(self, inputs, training=False):
        x = self.features(inputs, training=training)
        x = self.flatten(x)
        x = self.classifier(x)
        
        return x

In [7]:
model = ConvolutionalNeuralNetwork()
criterion = losses.SparseCategoricalCrossentropy()
optimizer = optimizers.Adam(learning_rate=ETA)

train_loss_obj = metrics.SparseCategoricalCrossentropy()
test_loss_obj = metrics.SparseCategoricalCrossentropy()

train_acc_obj = metrics.SparseCategoricalAccuracy()
test_acc_obj = metrics.SparseCategoricalAccuracy()

In [8]:
def train_step(x, y):
    
    with tf.GradientTape() as tape:
        ps = model(x)
        loss = criterion(y, ps)
        
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    train_loss_obj.update_state(y, ps)
    train_acc_obj.update_state(y, ps)

In [9]:
def test_step(x, y):
    
    ps = model(x)
    loss = criterion(y, ps)
    
    test_loss_obj.update_state(y, ps)
    test_acc_obj.update_state(y, ps)

In [10]:
def train():
    
    for e in range(EPOCHS):
        
        for x, y in trainloader:
            x = tf.cast(x, tf.float32)/256
            x = tf.expand_dims(x, axis=-1)
            train_step(x, y)
            
        for x, y in testloader:
            x = tf.cast(x, tf.float32)/256
            x = tf.expand_dims(x, axis=-1)
            test_step(x, y)
            
        train_loss = train_loss_obj.result()
        test_loss = test_loss_obj.result()
        
        train_acc = train_acc_obj.result()
        test_acc = test_acc_obj.result()
        
        print(f"Epochs {e+1}/{EPOCHS}, train loss: {train_loss:.8f}, train acc: {train_acc:.4f}, test loss: {test_loss:.8f}, test acc: {test_acc:.4f}")
        
        train_loss_obj.reset_states()
        test_loss_obj.reset_states()
        train_acc_obj.reset_states()
        test_acc_obj.reset_states()

In [11]:
train()

Epochs 1/10, train loss: 0.20986743, train acc: 0.9387, test loss: 0.07227330, test acc: 0.9768
Epochs 2/10, train loss: 0.06776577, train acc: 0.9795, test loss: 0.05704554, test acc: 0.9831
Epochs 3/10, train loss: 0.04428998, train acc: 0.9866, test loss: 0.04793566, test acc: 0.9843
Epochs 4/10, train loss: 0.03384862, train acc: 0.9893, test loss: 0.04617216, test acc: 0.9855
Epochs 5/10, train loss: 0.02655715, train acc: 0.9916, test loss: 0.05465016, test acc: 0.9829
Epochs 6/10, train loss: 0.01944853, train acc: 0.9935, test loss: 0.06646930, test acc: 0.9797
Epochs 7/10, train loss: 0.01642764, train acc: 0.9947, test loss: 0.05588628, test acc: 0.9842
Epochs 8/10, train loss: 0.01496342, train acc: 0.9949, test loss: 0.06000015, test acc: 0.9843
Epochs 9/10, train loss: 0.01087986, train acc: 0.9963, test loss: 0.06420957, test acc: 0.9840
Epochs 10/10, train loss: 0.00952641, train acc: 0.9970, test loss: 0.06586394, test acc: 0.9843
